In [72]:
import pandas as pd
import numpy as np
import pyodbc
from datetime import timedelta
from tqdm import tqdm
from scipy.stats import pearsonr

In [28]:
data_frame = pd.read_excel(r'data/StocksFrameIBOV.xlsx')
data_frame = data_frame.set_index('RptDt')
display(data_frame.head(3))

,TckrLst,ClsPrc,NmbrOfStcks
RptDt,,,
1997-09-06,"['ACE1', 'ACE2', 'ALP3', 'ALP4', 'ANH1', 'ANO5...","[9.0, 4.3, 435.0, 370.01, 8.51, 7.0, 590.0, 24...",128
1997-09-07,"['ACE1', 'ACE2', 'ALP3', 'ALP4', 'ANH1', 'ANO5...","[9.0, 4.8, 435.0, 379.99, 10.5, 7.0, 850.0, 26...",127
1997-09-08,"['ACE1', 'ACE2', 'ALP3', 'ALP4', 'ANH1', 'ANO5...","[10.0, 5.3, 450.0, 386.0, 10.5, 7.0, 650.0, 28...",127


------------
**Tratamento dos dados**

In [29]:
new_tickers = []
for i in tqdm(range(len(data_frame)), position=0, leave=True):
    df_to_list = data_frame.iloc[i][0].split(',')
    df_to_list[0] = df_to_list[0].replace('[',' ')
    df_to_list[-1] = df_to_list[-1].replace(']','')
    for j, element in enumerate(df_to_list):
        df_to_list[j] = element.replace(' ', '')
        df_to_list[j] = df_to_list[j][1:len(df_to_list[j])-1]
    new_tickers.append(df_to_list)

100%|████████████████████████████████████████████████████████████████████████████| 8881/8881 [00:03<00:00, 2578.32it/s]


In [30]:
new_prices = []
for i in tqdm(range(len(data_frame)), position=0, leave=True):
    df_to_list = data_frame.iloc[i][1].split(',')
    df_to_list[0] = df_to_list[0].replace('[',' ')
    df_to_list[-1] = df_to_list[-1].replace(']','')
    for j, element in enumerate(df_to_list):
        df_to_list[j] = element.replace(' ', '')
        df_to_list[j] = float(df_to_list[j])
    new_prices.append(df_to_list)

100%|████████████████████████████████████████████████████████████████████████████| 8881/8881 [00:03<00:00, 2519.27it/s]


In [31]:
data_frame.drop(['TckrLst', 'ClsPrc'], axis=1)
data_frame['TckrLst'] = new_tickers
data_frame['ClsPrc'] = new_prices
display(data_frame.head(3))

,TckrLst,ClsPrc,NmbrOfStcks
RptDt,,,
1997-09-06,"[ACE1, ACE2, ALP3, ALP4, ANH1, ANO5, ART2, AVI...","[9.0, 4.3, 435.0, 370.01, 8.51, 7.0, 590.0, 24...",128
1997-09-07,"[ACE1, ACE2, ALP3, ALP4, ANH1, ANO5, ART2, AVI...","[9.0, 4.8, 435.0, 379.99, 10.5, 7.0, 850.0, 26...",127
1997-09-08,"[ACE1, ACE2, ALP3, ALP4, ANH1, ANO5, ART2, AVI...","[10.0, 5.3, 450.0, 386.0, 10.5, 7.0, 650.0, 28...",127


-----------
**Intervalo de dias nos quais iremos estimar a correlação**

In [32]:
begin_date = pd.to_datetime('2000-09-06')
end_date = pd.to_datetime('2000-09-06')
print(f'Begin Date: {begin_date}')
print(f'End Date: {end_date}')

Begin Date: 2000-09-06 00:00:00
End Date: 2000-09-06 00:00:00


In [33]:
window = 365

In [34]:
#Capta os preços de acordo com os índices
def prices_stocks (data_frame, index_stock, start, stop):
    prices = []
    count = 0
    for k in range(data_frame.index.get_loc(start), data_frame.index.get_loc(stop)+1):
        prices.append(data_frame.iloc[k][1][index_stock[count]])
        count = count + 1
    return prices

In [35]:
#Estima as posições de cada ação nas listas de tickres no período de 1 ano
def prices_to_estimation (data_frame, intersection, start, stop, stock):
    index_stock = []
    for j in range(data_frame.index.get_loc(start), data_frame.index.get_loc(stop)+1):
        index_stock.append(data_frame.iloc[j][0].index(stock))
    prices = prices_stocks(data_frame, index_stock, start, stop)
    return prices

In [36]:
#Encontra a interceção entre duas listas
def common_elements (current_tickrs, previous_tickrs):
    intersection = []
    aux = []
    for element in current_tickrs:
        validation = [element in previous_tickrs[o] for o in range(len(previous_tickrs))]
        if all(validation):
            intersection.append(element)

    return intersection

In [37]:
for i in tqdm(range(data_frame.index.get_loc(begin_date), data_frame.index.get_loc(end_date)+1),\
              position=0, leave=True, desc = 'elementos comuns'):
    
    start = data_frame.index[i] -  timedelta(days=window)
    stop = data_frame.index[i]
    
    current_tickrs = data_frame.loc[stop]['TckrLst']
    previous_tickrs = data_frame.loc[start:stop]['TckrLst'].tolist()
    
    intersection = common_elements(current_tickrs, previous_tickrs)
    
    all_prices = []
    for stock in intersection:
        prices = prices_to_estimation (data_frame, intersection, start, stop, stock)
        all_prices.append(prices)

elementos comuns: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.64s/it]


In [70]:
stocks_dataframe = pd.DataFrame(np.array(all_prices).T, columns=intersection)

In [71]:
stocks_dataframe

,ALP4,AVI2,AZE2,BB2,BB3,BBD4,BEL1,BEL2,BES2,BRH2,...,RIO2,RPS2,SAG2,SCO4,SUZ2,TUP4,VAG2,VAL2,VSM1,WHM1
0,1395.00,12.30,18.99,930.0,656.0,170.0,280.0,177.00,15.50,170.0,...,18.01,280.0,26.50,21.0,2100.00,234.95,94.00,2850.0,780.00,17.49
1,1388.00,12.20,18.91,980.0,660.0,170.0,280.0,180.00,15.30,178.0,...,19.00,290.0,27.50,21.0,2099.99,240.00,93.55,2900.0,800.00,17.50
2,1388.00,12.75,18.26,1020.0,660.0,173.0,275.0,180.00,14.99,175.0,...,18.00,290.0,25.50,22.0,2000.00,250.00,92.49,3000.0,805.00,18.20
3,1388.00,11.80,18.70,1000.0,670.0,175.0,268.5,171.00,15.10,181.0,...,18.00,270.0,24.01,23.0,1999.99,260.00,90.00,2900.0,815.00,18.10
4,1650.01,11.71,18.00,955.0,675.0,175.0,260.0,170.00,15.00,175.0,...,17.00,255.0,25.00,23.0,1949.99,230.00,83.50,2870.0,790.00,17.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,5000.00,67.00,800.00,15300.0,11000.0,1000.0,15600.0,11050.00,227.00,3050.0,...,1350.00,26000.0,630.00,265.0,160000.00,4000.00,15300.00,28300.0,78000.00,128.00
362,4800.00,69.00,810.00,15700.0,11000.0,1050.0,15700.0,11000.00,225.00,3000.0,...,1500.00,25500.0,610.00,275.0,160000.00,4000.00,16000.00,28700.0,79999.99,131.00
363,4999.99,75.00,900.00,16700.0,11500.0,1100.0,17000.0,11600.00,230.00,3000.0,...,1600.00,25500.0,620.00,285.0,175000.00,5300.00,17500.00,30800.0,81000.01,150.00
364,5300.00,81.00,940.00,17550.0,13000.0,1195.0,17600.0,12500.01,245.00,3200.0,...,1600.00,29500.0,630.00,290.0,190000.00,4300.00,19100.00,33400.0,95000.00,160.00


-------
**Exportar dados para um database**

In [100]:
data_conection = ("Driver={SQL Server};"
                 "Server=DESKTOP-0EKP870;"
                 "Database=IndexDataBase;")
cnxn = pyodbc.connect(data_conection)
cursor = cnxn.cursor()

In [101]:
columnName = list(stocks_dataframe.columns)
columnDataType = ['FLOAT' for i in range (len(columnName))]
createTableStatement = 'USE IndexDataBase \n CREATE TABLE Table{} ('.format(1)
for i in range(len(columnDataType)):
    createTableStatement = createTableStatement + '\n' + columnName[i] + ' ' + columnDataType[i] + ','
createTableStatement = createTableStatement[:-1] + ' );'

In [102]:
InsertValuesTable = 'INSERT INTO Table{}('.format(1)
for i in range(len(columnName)):
    InsertValuesTable = InsertValuesTable + columnName[i] + ',' + ' '
InsertValuesTable = InsertValuesTable[:-2] + ')\n VALUES'

In [103]:
subset = stocks_dataframe[list(stocks_dataframe.columns)]
tuples = [tuple(x) for x in subset.to_numpy()]

In [104]:
values_rows = '('
for numeric in tuples[0]:
    values_rows = values_rows + str(numeric) + ',' + ' '
values_rows = values_rows[:-2] + ')' + ',' + '\n'
for i in tqdm(range(1, len(tuples)-1)):
    values_rows = values_rows + '('
    for numeric in tuples[i]:
        values_rows = values_rows + str(numeric) + ',' + ' '
    values_rows = values_rows[:-2] + ')' + ',' + '\n'
values_rows = values_rows + '('
for numeric in tuples[-1]:
    values_rows = values_rows + str(numeric) + ',' + ' '
values_rows = values_rows[:-2] + ')' + '\n'

100%|██████████████████████████████████████████████████████████████████████████████| 364/364 [00:00<00:00, 2401.15it/s]


In [105]:
cursor.execute(createTableStatement + InsertValuesTable + values_rows)
cnxn.commit()